In [2]:
from src.data.dataset_functions import get_dataframei
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
import functools
from pyspark.sql.types import *

dataFrame = get_dataframei(38)

providing ../data/raw/2022_place_canvas_history-000000000038.csv ...
../data/raw/2022_place_canvas_history-000000000038.csv is already in data/raw
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1b2cac7e-4c0f-4b5b-bc13-e0b2f3d0c87b;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 70ms :: artifacts dl 4ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------

In [2]:
timeDiffFrame = dataFrame.alias('df1').join(dataFrame.alias('df2'),F.col('df1.user_id') == F.col('df2.user_id'),'inner')
timeDiffFrame = timeDiffFrame.where('df2.t > df1.t')
timeDiffFrame = timeDiffFrame.orderBy(['df1.user_id', 'df1.t'])
timeDiffFrame = timeDiffFrame.select(F.col('df1.user_id'),F.col('df1.t').alias('t1'),F.col('df2.t').alias('t2'),(F.col('df2.t') - F.col('df1.t')).alias('diff'))
#timeDiffFrame.printSchema()

In [3]:
timeDiffReduced = timeDiffFrame.dropDuplicates(['user_id', 't1'])
timeDiffReduced = timeDiffReduced.orderBy(['user_id', 't1'])

In [4]:
botDiff = timeDiffReduced.alias('tdr1').join(timeDiffReduced.alias('tdr2'),F.col('tdr1.user_id') == F.col('tdr2.user_id'),'inner').join(timeDiffReduced.alias('tdr3'),F.col('tdr1.user_id') == F.col('tdr3.user_id'), 'inner').join(timeDiffReduced.alias('tdr4'),F.col('tdr1.user_id') == F.col('tdr4.user_id'), 'inner').join(timeDiffReduced.alias('tdr5'),F.col('tdr1.user_id') == F.col('tdr5.user_id'), 'inner')
botDiff = botDiff.where('tdr1.diff == tdr2.diff').where('tdr1.t2 == tdr2.t1').where('tdr3.diff == tdr1.diff').where('tdr2.t2 == tdr3.t1').where('tdr1.diff == tdr4.diff').where('tdr3.t2 == tdr4.t1').where('tdr1.diff == tdr5.diff').where('tdr4.t2 == tdr5.t1')
botDiff = botDiff.select('tdr1.user_id','tdr1.diff')
botDiff = botDiff.dropDuplicates(['user_id'])
botDiff = botDiff.orderBy(['user_id'])
#botDiff.printSchema()

In [5]:
botDiff = botDiff.drop("diff")
#botDiff.printSchema()

In [3]:
#bisherige Liste auslesen
spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()
df = spark.read.option("header",True) \
     .csv("../data/test_liste/fck_liste.csv")
#df = spark.read.csv("../data/test_liste/fck_liste.csv")
df.printSchema()

root
 |-- user_id: string (nullable = true)



In [7]:
#Jetzt beide DataFrames vergleichen und schauen welche IDs noch nicht in der Liste sind
joined = botDiff.alias('bots').join(df.alias('list'),F.col('bots.user_id') == F.col('list.user_id'),'fullouter')
#joined.drop('list.user_id')
#joined.show(5)

In [8]:
joined = joined.where("bots.user_id IS NOT NULL")
joined = joined.where("list.user_id IS NULL")
#joined.show(7)

In [9]:
#Duplikate eliminieren, zweite Spalte droppen
test = joined.select("bots.user_id").dropDuplicates(["user_id"])
#test.show(10)

In [10]:
#Ergänzung in csv liste packen
test.write.option("header",True).csv("../data/test_liste/fck_liste2")
#test.write.save(path='../data/test_liste/fck_liste.csv', format='csv', mode='append', sep='\t')

22/07/11 17:17:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/07/11 17:17:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/07/11 17:17:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/07/11 17:17:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


In [6]:
#jetzt die Liste in eine csv schreiben! (NUR INITIAL UM EINER ERSTE LISTE ZU KRIEGEN!)
#../reports/figures/mods-censors-canvas-df5.jpg
#botDiff.write.format("csv").save("../data/test/test_liste.csv")
botDiff.write.option("header",True).csv("../data/test_liste")

22/07/11 15:51:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/07/11 15:51:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/07/11 15:51:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/07/11 15:51:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/07/11 15:51:33 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/07/11 15:51:33 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/07/11 15:51:33 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/07/11 15:51:33 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


In [4]:
total = df.dropDuplicates(['user_id'])
total.write.option("header",True).csv("../data/test_liste/fck_liste2")